## Part I Interpretable Regression Methods

### set up a regression experiment

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
price = pd.read_csv('./processed-data/new_train.csv')

In [2]:
y = price['SalePrice']
X = price.drop(['SalePrice'],axis=1)
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

### explore the dataset

In [3]:
from interpret import show
from interpret.data import Marginal

marginal = Marginal().explain_data(X_train, y_train, name = 'Train Data')
show(marginal)

<!-- http://127.0.0.1:7320/140548708970904/ -->

### Train the Explainable Boosting Machine (EBM)

In [4]:
from interpret.glassbox import ExplainableBoostingRegressor, LinearRegression, RegressionTree

ebm = ExplainableBoostingRegressor(random_state=seed)
ebm.fit(X_train, y_train)

ExplainableBoostingRegressor(binning_strategy='quantile', data_n_episodes=2000,
                             early_stopping_run_length=50,
                             early_stopping_tolerance=1e-05,
                             feature_names=['MSSubClass', 'MSZoning',
                                            'LotFrontage', 'LotArea', 'Street',
                                            'Alley', 'LotShape', 'LandContour',
                                            'Utilities', 'LotConfig',
                                            'LandSlope', 'Neighborhood',
                                            'Condition1', 'Condition2',
                                            'BldgType', 'HouseStyle',
                                            'Overa...
                                            'categorical', 'categorical',
                                            'categorical', 'continuous',
                                            'continuous', 'continuous',
            

### Global Explanations: What the model learned overall

In [5]:
ebm_global = ebm.explain_global(name='EBM')
show(ebm_global)

<!-- http://127.0.0.1:7320/140548846693400/ -->

### Local Explanations: How an individual prediction was made

In [6]:
ebm_local = ebm.explain_local(X_test[:10], y_test[:10], name='EBM')
show(ebm_local)

<!-- http://127.0.0.1:7320/140548841905848/ -->

### Evaluate EBM performance

In [7]:
from interpret import show
from interpret.perf import RegressionPerf

ebm_perf = RegressionPerf(ebm.predict).explain_perf(X_test, y_test, name='EBM')
show(ebm_perf)

<!-- http://127.0.0.1:7320/140548840854528/ -->

### Let's test out a few other Explainable Models

In [8]:
X = pd.get_dummies(data=X, drop_first=True)
seed = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed)

lr = LinearRegression(random_state=seed)
lr.fit(X_train, y_train)

rt = RegressionTree(random_state=seed)
rt.fit(X_train, y_train)

/Users/scarlettwang/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 256703707677.99158, tolerance: 711570848.3106698



### Compare performance using the Dashboard

In [9]:
lr_perf = RegressionPerf(lr.predict).explain_perf(X_test, y_test, name='Linear Regression')
rt_perf = RegressionPerf(rt.predict).explain_perf(X_test, y_test, name='Regression Tree')

show(lr_perf)
show(rt_perf)
show(ebm_perf)

<!-- http://127.0.0.1:7320/140548839889664/ -->

<!-- http://127.0.0.1:7320/140548840008000/ -->

<!-- http://127.0.0.1:7320/140548840854528/ -->

## Part II Explaining Blackbox Regressors

### Train a blackbox regression system

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pca = PCA()
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)

blackbox_model = Pipeline([('pca', pca), ('rf', rf)])
blackbox_model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('rf',
                 RandomForestRegressor(bootstrap=True, criterion='mse',
                                       max_depth=None, max_features='auto',
                                       max_leaf_nodes=None,
                                       min_impurity_decrease=0.0,
                                       min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0,
                                       n_estimators=100, n_jobs=-1,
                                       oob_score=False, random_state=None,
                                       verbose=0, warm_start=False))],
         verbose=False)

### Show blackbox model performance

In [11]:
from interpret import show
from interpret.perf import RegressionPerf

blackbox_perf = RegressionPerf(blackbox_model.predict).explain_perf(X_test, y_test, name='Blackbox')
show(blackbox_perf)

<!-- http://127.0.0.1:7320/140548839887984/ -->

### Local Explanations: How an individual prediction was made

In [12]:
from interpret.blackbox import LimeTabular
from interpret import show

lime = LimeTabular(predict_fn=blackbox_model.predict, data=X_train, random_state=1)

lime_local = lime.explain_local(X_test[:5], y_test[:5], name='LIME')

show(lime_local)

<!-- http://127.0.0.1:7320/140548366090192/ -->

In [13]:
from interpret.blackbox import ShapKernel
import numpy as np
feature_names = X_train.columns.tolist()
background_val = np.median(X_train, axis=0).reshape(1, -1)
shap = ShapKernel(predict_fn=blackbox_model.predict, data=background_val,feature_names = feature_names)
shap_local = shap.explain_local(X_test[:10], y_test[:10], name='SHAP')
show(shap_local)

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!
l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_

l1_reg="auto" is deprecated and in the next version (v0.29) the behavior will change from a conditional use of AIC to simply "num_features(10)"!


<!-- http://127.0.0.1:7320/140548857335936/ -->

### Global Explanations: How the model behaves overal

In [14]:
from interpret.blackbox import MorrisSensitivity

sensitivity = MorrisSensitivity(predict_fn=blackbox_model.predict, data=X_train)
sensitivity_global = sensitivity.explain_global(name="Global Sensitivity")

show(sensitivity_global)

<!-- http://127.0.0.1:7320/140548407231376/ -->

In [15]:
from interpret.blackbox import PartialDependence

pdp = PartialDependence(predict_fn=blackbox_model.predict, data=X_train)
pdp_global = pdp.explain_global(name='Partial Dependence')

show(pdp_global)

<!-- http://127.0.0.1:7320/140546458838072/ -->

### Compare them all in the Dashboard

In [ ]:
show([blackbox_perf, lime_local, shap_local, sensitivity_global, pdp_global])